In [2]:
import numpy as np
from matplotlib import pyplot as plt
from pyESN.pyESN import ESN
from utils import *
from scipy.signal import periodogram,decimate

In [16]:
# eeg = get_eeg_by_fileindex(1)
metadata = read_metadata_by_fileindex(1)
# eeg = decimate(eeg, q=10).reshape(-1, 1)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
eeg = metadata.iloc[:,2:-2]
ctrl = scaler.fit_transform(eeg).astype(float)
output = metadata['label'].to_numpy().astype(float)

In [ ]:
ctrl = np.zeros((1,501))
output = np.zeros((1,1))
from tqdm import tqdm
for i,epoch_label in tqdm(enumerate(metadata['label'].to_numpy())):
    ctrl = np.vstack((ctrl,np.hstack((1,eeg[i*500:(i*500)+500]))))
    output = np.vstack((output,epoch_label))
ctrl = np.delete(ctrl,0,axis=0)
output = np.delete(output,0,axis=0)

In [17]:
N = len(ctrl) # signal length
traintest_cutoff = int(np.ceil(0.5*N))

In [18]:
train_ctrl,train_output = ctrl[:traintest_cutoff],output[:traintest_cutoff]
test_ctrl, test_output  = ctrl[traintest_cutoff:],output[traintest_cutoff:]

In [20]:
esn = ESN(n_inputs = 40,
        n_outputs = 1,
        n_reservoir = 200,
        spectral_radius = .25,
        sparsity = 0.95,
        noise = 0.001,
        out_activation = np.tanh,
        inverse_out_activation = np.arctanh,
        silent = False)

pred_train = esn.fit(train_ctrl,train_output)

# print("test error:")
# pred_test = esn.predict(test_ctrl)
# print(np.sqrt(np.mean((pred_test - test_output)**2)))

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(10,1.5))
plt.plot(train_ctrl[:,1],label='control')
plt.plot(train_output,label='target')
plt.plot(pred_train,label='model')
plt.legend(fontsize='x-small')
plt.title('training (excerpt)')

plt.figure(figsize=(10,4))
plt.plot(test_ctrl[:,1],label='control')
plt.plot(test_output,label='target')
plt.plot(pred_test,label='model')
plt.legend(fontsize='x-small')
plt.title('test (excerpt)')


In [ ]:
df = pd.DataFrame(W)
ws = []
df = df[df!=0]
for col in df.columns:
    for weight in df[col].dropna().to_numpy():
        ws.append(weight)
plt.hist(ws)